In [78]:
import pandas as pd
from pandas.stats.api import ols
from sklearn import datasets, linear_model
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
import math
%matplotlib inline
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.cross_validation import train_test_split
import math

from sklearn.cross_validation import cross_val_score, cross_val_predict
from sklearn import metrics

filepath = '/Users/eshwarchandrasekharan/Desktop/repo/predict-pop/code/'
# train_df.shape, train_df[~(train_df.content_id == -1)].shape

In [79]:
features = [
       'next_external_id', 
       'external_id', 
       'display_page', 
       'firsthour_consumptions_by_type__link_clicks',
       'firsthour_stories_by_action_type__share',
       'firsthour_video_views_10s_organic',
       'firsthour_video_complete_views_30s_organic',
       'firsthour_video_complete_views_organic',
       'twodays_consumptions_by_type__link_clicks',
       'twodays_stories_by_action_type__share',
       'twodays_video_views_10s_organic',
       'twodays_video_complete_views_30s_organic',
       'twodays_video_complete_views_organic',
        ]

one_hour_features = [
                  'firsthour_consumptions_by_type__link_clicks',
       'firsthour_stories_by_action_type__share',
       'firsthour_video_views_10s_organic',
       'firsthour_video_complete_views_30s_organic',
       'firsthour_video_complete_views_organic',
                ]

two_day_features = [
        'twodays_consumptions_by_type__link_clicks',
       'twodays_stories_by_action_type__share',
       'twodays_video_views_10s_organic',
       'twodays_video_complete_views_30s_organic',
       'twodays_video_complete_views_organic',
    ]

# cross_promotion_features = all_pages

In [80]:
from sklearn import linear_model
clf = linear_model.LinearRegression()

print("LINEAR REGRESSION!")

train_df = pd.read_csv('previous-to-next-page-info-videos-jun16.csv')

###restrict post_type for videos
# train_df = train_df[train_df['post_type_id'] == 2]

# train_df = pd.read_csv('previous-to-next-page-info-vids.csv')
# train_df = pd.read_csv(filepath+'previous-to-next-page-info.csv')
all_pages = list(train_df.display_page.unique())
cross_promotion_features = all_pages

print("With log-scaled performance stats, and also cross-promotion features and page infos!")

page_infos = ['display_page', 'next_page']

# train_features = one_hour_features
# train_features = one_hour_features + page_infos
train_features = one_hour_features + page_infos + cross_promotion_features
# train_features = cross_promotion_features + one_hour_features
# train_features = two_day_features + one_hour_features + cross_promotion_features

for feats in (one_hour_features):
    train_df[feats] = np.log(train_df[feats] + 1)

# lm = linear_model.LinearRegression(fit_intercept=True, normalize=True)
cv = 10

train_df['clicks_bucket'] = np.log(train_df['next_clicks'] + 1).astype(int)
train_df['share_bucket'] = np.log(train_df['next_shares']+1).astype(int)

print("No. of data-points = ", len(train_df))

# y_clicks = train_df['next_clicks']
# y_share = train_df['next_shares']
y_clicks = train_df['clicks_bucket']
y_share = train_df['share_bucket']

###generate DFs for analysis - X and Y
X = train_df[train_features]
####convert page_infos into dummies
X = pd.get_dummies(data = X, columns=['display_page', 'next_page'])

from sklearn.model_selection import KFold

fold = 0
cv_folds = 10
kf = KFold(n_splits = cv_folds, shuffle = True)

clicks_accuracy = []
clicks_error = []
shares_accuracy = []
shares_error = []

for train_index, test_index in kf.split(X):
#     print("Fold = ", fold)
#     print("Shares: Fold = ", fold)
    y = y_share
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]
    clf.fit(X, y)
    y_pred = clf.predict(X_test)
        
    accuracy = metrics.r2_score(y_test, y_pred)
#     print("Cross-Predicted Accuracy (R2):", accuracy)
    shares_accuracy.append(accuracy)
    from sklearn.metrics import mean_absolute_error
    # print("Mean Absolute Error: ", mean_absolute_error(y, predictions))
    error_percent = mean_absolute_error(y_test, y_pred)/y.mean()
#     print("Mean values (share): ", y_test.mean(), " | percent error: ",  error_percent)
    shares_error.append(error_percent)
    
#     print("Clicks: Fold = ", fold)
    y = y_clicks
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]
    clf.fit(X, y)
    y_pred = clf.predict(X_test)
    
    accuracy = metrics.r2_score(y_test, y_pred)
#     print("Cross-Predicted Accuracy (R2):", accuracy)
    clicks_accuracy.append(accuracy)
    from sklearn.metrics import mean_absolute_error
    # print("Mean Absolute Error: ", mean_absolute_error(y, predictions))
    error_percent = mean_absolute_error(y_test, y_pred)/y.mean()
#     print("Mean values (share): ", y_test.mean(), " | percent error: ",  error_percent)
    clicks_error.append(error_percent)
    fold += 1
print("Shares performance: Accuracy = ", np.mean(shares_accuracy), " ; Error (/100) = ", np.mean(shares_error))
print("Clicks performance: Accuracy = ", np.mean(clicks_accuracy), " ; Error (/100) = ", np.mean(clicks_error))

LINEAR REGRESSION!
With log-scaled performance stats, and also cross-promotion features and page infos!
No. of data-points =  2035
Shares performance: Accuracy =  0.420337696079  ; Error (/100) =  1.11310665282
Clicks performance: Accuracy =  0.847513652772  ; Error (/100) =  0.205109135404


In [66]:
print("ONLY 1 HOUR PERFORMANCE IN PREVIOUS PAGE")
print('SHARES:')
res = ols(y = y_share, x = X)
print(res)
print('CLICKS:')
res = ols(y =y_clicks, x = X)
print(res)

ONLY 1 HOUR PERFORMANCE IN PREVIOUS PAGE
SHARES:

-------------------------Summary of Regression Analysis-------------------------

Formula: Y ~ <firsthour_consumptions_by_type__link_clicks>
             + <firsthour_stories_by_action_type__share> + <firsthour_video_views_10s_organic>
             + <firsthour_video_complete_views_30s_organic>
             + <firsthour_video_complete_views_organic> + <BuzzFeed Video> + <Bring Me> + <BuzzFeed France>
             + <BuzzFeed Style> + <Nifty Pets> + <BuzzFeed Parents>
             + <BuzzFeed Japan> + <Tasty> + <BuzzFeed Justin> + <BuzzFeed Violet>
             + <BuzzFeed News> + <BuzzFeed Ryan> + <BuzzFeed Unsolved> + <SOML> + <Tasty Miam>
             + <Proper Tasty> + <Top Knot> + <BuzzFeed Robin> + <Tasty Japan>
             + <BuzzFeed UK News> + <BuzzFeed UK> + <BuzzFeed Politics>
             + <Cocoa Butter> + <BuzzFeed UK Politics> + <Goodful> + <BuzzFeed Food>
             + <BuzzFeed IRL> + <BuzzFeed> + <BuzzFeed Animals> + 

/Users/eshwarchandrasekharan/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2881: FutureWarning: The pandas.stats.ols module is deprecated and will be removed in a future version. We refer to external packages like statsmodels, see some examples here: http://www.statsmodels.org/stable/regression.html
  exec(code_obj, self.user_global_ns, self.user_ns)
/Users/eshwarchandrasekharan/anaconda3/lib/python3.6/site-packages/pandas/stats/ols.py:336: RuntimeWarning: invalid value encountered in sqrt
  return np.sqrt(np.diag(self._var_beta_raw))
/Users/eshwarchandrasekharan/anaconda3/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:875: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
/Users/eshwarchandrasekharan/anaconda3/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:875: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
/Users/eshwarchandrasekharan/anaconda3/



-------------------------Summary of Regression Analysis-------------------------

Formula: Y ~ <firsthour_consumptions_by_type__link_clicks>
             + <firsthour_stories_by_action_type__share> + <firsthour_video_views_10s_organic>
             + <firsthour_video_complete_views_30s_organic>
             + <firsthour_video_complete_views_organic> + <BuzzFeed Video> + <Bring Me> + <BuzzFeed France>
             + <BuzzFeed Style> + <Nifty Pets> + <BuzzFeed Parents>
             + <BuzzFeed Japan> + <Tasty> + <BuzzFeed Justin> + <BuzzFeed Violet>
             + <BuzzFeed News> + <BuzzFeed Ryan> + <BuzzFeed Unsolved> + <SOML> + <Tasty Miam>
             + <Proper Tasty> + <Top Knot> + <BuzzFeed Robin> + <Tasty Japan>
             + <BuzzFeed UK News> + <BuzzFeed UK> + <BuzzFeed Politics>
             + <Cocoa Butter> + <BuzzFeed UK Politics> + <Goodful> + <BuzzFeed Food>
             + <BuzzFeed IRL> + <BuzzFeed> + <BuzzFeed Animals> + <BuzzFeed México>
             + <Tasty Junior> 

In [81]:
# from sklearn import linear_model
# clf = linear_model.LinearRegression()
# print("LINEAR REGRESSION!")

from sklearn.tree import DecisionTreeRegressor
clf = DecisionTreeRegressor(max_depth = 10)
print("TREE REGRESSION!")

train_df = pd.read_csv('previous-to-next-page-info-videos-jun16.csv')
###restrict post_type for videos
# train_df = train_df[train_df['post_type_id'] == 16]

# train_df = pd.read_csv('previous-to-next-page-info-vids.csv')
all_pages = list(train_df.display_page.unique())

print("Raw counts of performance stats with page names included!")
# print("WITH cross-promotion features")

page_infos = ['display_page', 'next_page']

# train_features = one_hour_features
train_features = one_hour_features + page_infos + cross_promotion_features
# train_features = one_hour_features + two_day_features + page_infos
# train_features = cross_promotion_features + one_hour_features

# train_features = two_day_features + one_hour_features + cross_promotion_features

# for feats in (one_hour_features):
#     train_df[feats] = np.log(train_df[feats] + 1)

# lm = linear_model.LinearRegression(fit_intercept=True, normalize=True)
cv = 10

train_df['clicks_bucket'] = np.log(train_df['next_clicks'] + 1).astype(int)
train_df['share_bucket'] = np.log(train_df['next_shares']+1).astype(int)

print("No. of data-points = ", len(train_df))

y_clicks = train_df['next_clicks']
y_share = train_df['next_shares']
# y_clicks = train_df['clicks_bucket']
# y_share = train_df['share_bucket']

###generate DFs for analysis - X and Y
X = train_df[train_features]
####convert page_infos into dummies
X = pd.get_dummies(data = X, columns=['display_page', 'next_page'])

from sklearn.model_selection import KFold

fold = 0
cv_folds = 10
kf = KFold(n_splits = cv_folds, shuffle = True)

clicks_accuracy = []
clicks_error = []
shares_accuracy = []
shares_error = []

for train_index, test_index in kf.split(X):
#     print("Fold = ", fold)
#     print("Shares: Fold = ", fold)
    y = y_share
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]
    clf.fit(X, y)
    y_pred = clf.predict(X_test)
        
    accuracy = metrics.r2_score(y_test, y_pred)
#     print("Cross-Predicted Accuracy (R2):", accuracy)
    shares_accuracy.append(accuracy)
    from sklearn.metrics import mean_absolute_error
    # print("Mean Absolute Error: ", mean_absolute_error(y, predictions))
    error_percent = mean_absolute_error(y_test, y_pred)/y.mean()
#     print("Mean values (share): ", y_test.mean(), " | percent error: ",  error_percent)
    shares_error.append(error_percent)
    
#     print("Clicks: Fold = ", fold)
    y = y_clicks
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]
    clf.fit(X, y)
    y_pred = clf.predict(X_test)
    
    accuracy = metrics.r2_score(y_test, y_pred)
#     print("Cross-Predicted Accuracy (R2):", accuracy)
    clicks_accuracy.append(accuracy)
    from sklearn.metrics import mean_absolute_error
    # print("Mean Absolute Error: ", mean_absolute_error(y, predictions))
    error_percent = mean_absolute_error(y_test, y_pred)/y.mean()
#     print("Mean values (share): ", y_test.mean(), " | percent error: ",  error_percent)
    clicks_error.append(error_percent)
    fold += 1
print("Shares performance: Accuracy = ", np.mean(shares_accuracy), " ; Error (/100) = ", np.mean(shares_error))
print("Clicks performance: Accuracy = ", np.mean(clicks_accuracy), " ; Error (/100) = ", np.mean(clicks_error))

TREE REGRESSION!
Raw counts of performance stats with page names included!
No. of data-points =  2035
Shares performance: Accuracy =  0.733545970082  ; Error (/100) =  0.469037665792
Clicks performance: Accuracy =  0.888815323123  ; Error (/100) =  0.291574806303


In [82]:
print("ONLY 1 HOUR PERFORMANCE IN PREVIOUS PAGE")
print('SHARES:')
res = ols(y = y_share, x = X)
print(res)
print('CLICKS:')
res = ols(y =y_clicks, x = X)
print(res)

ONLY 1 HOUR PERFORMANCE IN PREVIOUS PAGE
SHARES:

-------------------------Summary of Regression Analysis-------------------------

Formula: Y ~ <firsthour_consumptions_by_type__link_clicks>
             + <firsthour_stories_by_action_type__share> + <firsthour_video_views_10s_organic>
             + <firsthour_video_complete_views_30s_organic>
             + <firsthour_video_complete_views_organic> + <BuzzFeed Video> + <Bring Me> + <BuzzFeed France>
             + <BuzzFeed Style> + <Nifty Pets> + <BuzzFeed Parents>
             + <BuzzFeed Japan> + <Tasty> + <BuzzFeed Justin> + <BuzzFeed Violet>
             + <BuzzFeed News> + <BuzzFeed Ryan> + <BuzzFeed Unsolved> + <SOML> + <Tasty Miam>
             + <Proper Tasty> + <Top Knot> + <BuzzFeed Robin> + <Tasty Japan>
             + <BuzzFeed UK News> + <BuzzFeed UK> + <BuzzFeed Politics>
             + <Cocoa Butter> + <BuzzFeed UK Politics> + <Goodful> + <BuzzFeed Food>
             + <BuzzFeed IRL> + <BuzzFeed> + <BuzzFeed Animals> + 

/Users/eshwarchandrasekharan/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2881: FutureWarning: The pandas.stats.ols module is deprecated and will be removed in a future version. We refer to external packages like statsmodels, see some examples here: http://www.statsmodels.org/stable/regression.html
  exec(code_obj, self.user_global_ns, self.user_ns)
/Users/eshwarchandrasekharan/anaconda3/lib/python3.6/site-packages/pandas/stats/ols.py:336: RuntimeWarning: invalid value encountered in sqrt
  return np.sqrt(np.diag(self._var_beta_raw))
/Users/eshwarchandrasekharan/anaconda3/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:875: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
/Users/eshwarchandrasekharan/anaconda3/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:875: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
/Users/eshwarchandrasekharan/anaconda3/


-------------------------Summary of Regression Analysis-------------------------

Formula: Y ~ <firsthour_consumptions_by_type__link_clicks>
             + <firsthour_stories_by_action_type__share> + <firsthour_video_views_10s_organic>
             + <firsthour_video_complete_views_30s_organic>
             + <firsthour_video_complete_views_organic> + <BuzzFeed Video> + <Bring Me> + <BuzzFeed France>
             + <BuzzFeed Style> + <Nifty Pets> + <BuzzFeed Parents>
             + <BuzzFeed Japan> + <Tasty> + <BuzzFeed Justin> + <BuzzFeed Violet>
             + <BuzzFeed News> + <BuzzFeed Ryan> + <BuzzFeed Unsolved> + <SOML> + <Tasty Miam>
             + <Proper Tasty> + <Top Knot> + <BuzzFeed Robin> + <Tasty Japan>
             + <BuzzFeed UK News> + <BuzzFeed UK> + <BuzzFeed Politics>
             + <Cocoa Butter> + <BuzzFeed UK Politics> + <Goodful> + <BuzzFeed Food>
             + <BuzzFeed IRL> + <BuzzFeed> + <BuzzFeed Animals> + <BuzzFeed México>
             + <Tasty Junior> +

In [62]:
y_share.mean(), y_clicks.mean()

(164.1031941031941, 502.12334152334154)

In [63]:
np.sqrt(1294.8886)/y_share.mean(), np.sqrt(750.5447)/y_clicks.mean()

(0.21928007419213424, 0.054560440873062929)

In [64]:
y_pred

array([  710.23295455,    26.73855544,    26.73855544,    26.73855544,
          26.73855544,    26.73855544,   572.74468085,    26.73855544,
          26.73855544,    42.66666667,    26.73855544,    26.73855544,
         710.23295455,    42.66666667,   710.23295455,  2277.525     ,
          26.73855544,   394.98701299,   710.23295455,    26.73855544,
         182.61111111,  3640.        ,   710.23295455,   710.23295455,
        1251.        ,  1103.08333333,  4310.        ,   621.44444444,
          42.66666667,    42.66666667,   968.5       ,    26.73855544,
         710.23295455,   710.23295455,   710.23295455,   394.98701299,
         460.        ,   394.98701299,  1596.        ,    26.73855544,
         162.        ,    26.73855544,    26.73855544,    26.73855544,
         162.        ,    26.73855544,    26.73855544,    26.73855544,
          26.73855544,   394.98701299,    26.73855544,    26.73855544,
        1042.27272727,    26.73855544,    26.73855544,    26.73855544,
      

In [67]:
train_df

,next_external_id,external_id,post_type_id,created_at,display_page,start_stats_date,firsthour_stats_date,firsthour_consumptions_by_type__link_clicks,firsthour_stories_by_action_type__share,firsthour_video_views_10s_organic,...,BuzzFeed Jazzmyne,Bored AF,BuzzFeed Sara,BuzzFeed Open Mic,This Is You,next_page,next_clicks,next_shares,clicks_bucket,share_bucket
0,1318800798260799_2240508729423330,1318800798260799_2240406312766905,16,2017-06-12 14:44:03.968914,BuzzFeed Video,2017-06-12 14:45:04 UTC,2017-06-12 15:45:09 UTC,6.950815,0.000000,11.435958,...,122.0,122.0,122.0,122.0,122.0,BuzzFeed Video,899.0,0.0,6,0
1,375093055847509_1482596015097202,1318800798260799_2231541953653341,2,2017-06-02 21:00:20.506696,BuzzFeed Video,2017-06-02 21:05:10 UTC,2017-06-02 22:05:12 UTC,6.424869,6.439350,11.553386,...,122.0,122.0,122.0,122.0,122.0,BuzzFeed Sweaty,0.0,60.0,0,4
2,1318800798260799_2240406312766905,1466985283336295_1627898583911630,2,2017-06-10 17:50:55.750101,Bring Me,2017-06-10 18:05:32 UTC,2017-06-10 19:05:12 UTC,5.105945,8.155075,11.645041,...,122.0,122.0,122.0,122.0,122.0,BuzzFeed Video,1043.0,0.0,6,0
3,196206360747821_441054412929680,160400084153937_651492571711350,2,2017-06-07 10:00:13.993119,BuzzFeed France,2017-06-07 10:05:07 UTC,2017-06-07 11:05:10 UTC,2.197225,3.663562,9.479680,...,122.0,122.0,122.0,122.0,122.0,Tasty Miam,0.0,0.0,0,0
4,1737181656494507_1931281730417831,1318800798260799_2237006066440263,16,2017-06-08 20:00:12.749271,BuzzFeed Video,2017-06-08 20:05:07 UTC,2017-06-08 21:05:05 UTC,6.115892,0.000000,10.219830,...,122.0,122.0,122.0,122.0,122.0,Proper Tasty,329.0,1251.0,5,7
5,294474594013113_1255496307910932,254018121357088_1391875557571333,2,2017-06-10 19:00:51.236926,BuzzFeed Style,2017-06-10 19:05:11 UTC,2017-06-10 20:05:09 UTC,3.044522,3.850148,8.345930,...,122.0,122.0,122.0,122.0,122.0,BuzzFeed DIY,60.0,0.0,4,0
6,685302218232254_1340187412743728,1318800798260799_2238237306317139,2,2017-06-12 00:01:09.432549,BuzzFeed Video,2017-06-12 00:05:12 UTC,2017-06-12 01:05:08 UTC,6.054439,5.298317,10.429990,...,122.0,122.0,122.0,122.0,122.0,BuzzFeed Eugene,0.0,0.0,0,0
7,1679616822293043_1888982188023171,1855438174733919_1916871461923923,2,2017-06-09 00:27:15.332341,Nifty Pets,2017-06-09 00:30:15 UTC,2017-06-09 01:30:06 UTC,4.043051,6.102559,9.610726,...,122.0,122.0,122.0,122.0,122.0,Nifty,501.0,2201.0,6,7
8,1318800798260799_2239812632826273,229476530583896_624786184386260,2,2017-06-11 14:30:12.330700,BuzzFeed Parents,2017-06-11 14:35:09 UTC,2017-06-11 15:35:04 UTC,3.091042,3.637586,7.035269,...,122.0,122.0,122.0,122.0,122.0,BuzzFeed Video,377.0,0.0,5,0
9,800616763324275_1565190483533562,1318800798260799_2238322682975268,16,2017-06-10 20:00:09.171064,BuzzFeed Video,2017-06-10 20:05:09 UTC,2017-06-10 21:05:08 UTC,5.730100,0.000000,9.388403,...,122.0,122.0,122.0,122.0,122.0,BuzzFeed Health,22.0,0.0,3,0


In [77]:
train_df['cross_promotion_pages']

0                           ['Bring Me' 'BuzzFeed Video']
1                    ['BuzzFeed Video' 'BuzzFeed Sweaty']
2                           ['Bring Me' 'BuzzFeed Video']
3                        ['BuzzFeed France' 'Tasty Miam']
4               ['Tasty' 'BuzzFeed Video' 'Proper Tasty']
5                       ['BuzzFeed Style' 'BuzzFeed DIY']
6                    ['BuzzFeed Video' 'BuzzFeed Eugene']
7                                  ['Nifty Pets' 'Nifty']
8                   ['BuzzFeed Parents' 'BuzzFeed Video']
9       ['Goodful' 'Tasty' 'Tasty Vegetarian' 'BuzzFee...
10                       ['Tasty Japan' 'BuzzFeed Japan']
11                               ['Tasty' 'Proper Tasty']
12      ['BuzzFeed Violet' 'BuzzFeed Justin' 'BuzzFeed...
13      ['Goodful' 'Tasty' 'Tasty Vegetarian' 'BuzzFee...
14      ['BuzzFeed Violet' 'BuzzFeed Justin' 'BuzzFeed...
15               ['BuzzFeed News' 'BuzzFeed Philippines']
16      ['BuzzFeed Unsolved' 'BuzzFeed Ryan' 'BuzzFeed...
17      ['Buzz